In [1]:
import pandas as pd  
import pickle 
import numpy as np 
from sqlalchemy import create_engine
import warnings
warnings.simplefilter(action='ignore')
import datetime 
import math 

In [2]:
hr_df= pd.read_pickle('heart_pickle.pkl')


In [3]:
selected_columns = ['_MICHD', 'SEXVAR','_AGE80', '_IMPRACE', '_BMI5',
                    '_RFBMI5','DIABETE4', 'CVDSTRK3', 'SMOKE100', 
                    '_RFHYPE6', 'TOLDHI3', '_RFDRHV7', 'EXERANY2', 
                   '_FRTLT1A', '_VEGLT1A']

In [4]:
hr_df = hr_df[selected_columns]
hr_df.head()

,_MICHD,SEXVAR,_AGE80,_IMPRACE,_BMI5,_RFBMI5,DIABETE4,CVDSTRK3,SMOKE100,_RFHYPE6,TOLDHI3,_RFDRHV7,EXERANY2,_FRTLT1A,_VEGLT1A
0,2.0,2.0,70.0,1.0,1454.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0
1,1.0,2.0,67.0,2.0,NaN,9.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0
2,1.0,2.0,72.0,2.0,2829.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0
3,2.0,2.0,62.0,1.0,3347.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,76.0,6.0,2873.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0


In [5]:
hr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438693 entries, 0 to 438692
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   _MICHD    434058 non-null  float64
 1   SEXVAR    438693 non-null  float64
 2   _AGE80    438693 non-null  float64
 3   _IMPRACE  438693 non-null  float64
 4   _BMI5     391841 non-null  float64
 5   _RFBMI5   438693 non-null  float64
 6   DIABETE4  438690 non-null  float64
 7   CVDSTRK3  438691 non-null  float64
 8   SMOKE100  417461 non-null  float64
 9   _RFHYPE6  438693 non-null  float64
 10  TOLDHI3   377857 non-null  float64
 11  _RFDRHV7  438693 non-null  float64
 12  EXERANY2  438691 non-null  float64
 13  _FRTLT1A  438693 non-null  float64
 14  _VEGLT1A  438693 non-null  float64
dtypes: float64(15)
memory usage: 50.2 MB


Clean Data, Replace Values to Build Model- readability 

In [6]:
hr_df.isnull().sum()

_MICHD       4635
SEXVAR          0
_AGE80          0
_IMPRACE        0
_BMI5       46852
_RFBMI5         0
DIABETE4        3
CVDSTRK3        2
SMOKE100    21232
_RFHYPE6        0
TOLDHI3     60836
_RFDRHV7        0
EXERANY2        2
_FRTLT1A        0
_VEGLT1A        0
dtype: int64

In [7]:
hr_df.duplicated().sum()

48772

In [8]:
hr_df = hr_df.drop_duplicates()

In [9]:
hr_df.shape

(389921, 15)

In [10]:
# referring to codebook: removed surveys where responses 
#were don't know (7)  & refused to answer (9)

In [11]:
#DIABETE4
hr_df = hr_df[hr_df['DIABETE4'] != 7]
hr_df = hr_df[hr_df['DIABETE4'] != 9]

#CVDSTRK3
hr_df = hr_df[hr_df['CVDSTRK3'] != 7]
hr_df = hr_df[hr_df['CVDSTRK3'] != 9]

#SMOKE100
hr_df = hr_df[hr_df['SMOKE100'] != 7]
hr_df = hr_df[hr_df['SMOKE100'] != 9]

#EXERANY2
hr_df = hr_df[hr_df['EXERANY2'] != 7]
hr_df = hr_df[hr_df['EXERANY2'] != 9]

#TOLDHI3
hr_df = hr_df[hr_df['TOLDHI3'] != 7]
hr_df = hr_df[hr_df['TOLDHI3'] != 9]

In [12]:
def replace_1(col): 

    col.replace({2:0}, inplace = True)
    col.fillna(col.mode()[0], inplace = True)
    return col

hr_df['_MICHD'] = (replace_1(hr_df['_MICHD']))
hr_df['SMOKE100'] = (replace_1(hr_df['SMOKE100']))
hr_df['TOLDHI3'] = (replace_1(hr_df['TOLDHI3']))
hr_df['EXERANY2'] = (replace_1(hr_df['EXERANY2']))
hr_df['CVDSTRK3'] = (replace_1(hr_df['CVDSTRK3']))

In [13]:
def replace_2(col): 
    col.replace({2:0}, inplace = True)
    col.replace({9:col.mode()[0]}, inplace = True)
    return col

hr_df['_FRTLT1A'] = (replace_2(hr_df['_FRTLT1A']))
hr_df['_VEGLT1A'] = (replace_2(hr_df['_VEGLT1A']))
hr_df['_RFHYPE6'] = (replace_2(hr_df['_RFHYPE6']))


In [14]:
#Diabetes
hr_df['DIABETE4'].replace({2:1, 3:0, 4:0}, inplace = True)
hr_df['DIABETE4'].fillna(hr_df['DIABETE4'].mode()[0], inplace = True)


In [17]:
def replace_3(col): 
    col.replace({1:0, 2:1, 3:2, 4:3, 5:4, 6:5}, inplace = True)
    return col

#hr_df['_AGE_G'] = (replace_3(hr_df['_AGE_G']))
hr_df['_IMPRACE'] = (replace_3(hr_df['_IMPRACE']))

In [18]:
def replace_4(col):

    col.replace({1:0, 2:1}, inplace = True)

    if 9 in col: 
        col.replace({9:col.mode()[0]}, inplace = True)
    return col
hr_df['SEXVAR'] = (replace_4(hr_df['SEXVAR']))
hr_df['_RFBMI5'] = (replace_4(hr_df['_RFBMI5']))
hr_df['_RFDRHV7'] = (replace_4(hr_df['_RFDRHV7']))

In [19]:
#BMI
hr_df['_BMI5']= hr_df['_BMI5'].div(100).round(0)
hr_df['_BMI5'].fillna(hr_df['_BMI5'].median(), inplace=True)

In [20]:
hr_df.isnull().sum()

_MICHD      0
SEXVAR      0
_AGE80      0
_IMPRACE    0
_BMI5       0
_RFBMI5     0
DIABETE4    0
CVDSTRK3    0
SMOKE100    0
_RFHYPE6    0
TOLDHI3     0
_RFDRHV7    0
EXERANY2    0
_FRTLT1A    0
_VEGLT1A    0
dtype: int64

In [21]:
#rename columns for readability 

hr_df = hr_df.rename(columns = {'_MICHD': 'heart_attack_disease', 
                               'SEXVAR': 'sex', 
                               '_AGE80':'age', 
                               '_IMPRACE': 'race_ethn', 
                               '_BMI5':'bmi', 
                               '_RFBMI5': 'overweight', 
                               'DIABETE4':'diabetes', 
                               'CVDSTRK3':'diagnose_strk', 
                               'SMOKE100':'smoker', 
                               '_RFHYPE6': 'high_bp', 
                               'TOLDHI3': 'high_chol', 
                               '_RFDRHV7': 'heavy_drinker', 
                               'EXERANY2': 'exercise', 
                               '_FRTLT1A': 'fruit',
                               '_VEGLT1A': 'vege'})

In [22]:
hr_df.columns

Index(['heart_attack_disease', 'sex', 'age', 'race_ethn', 'bmi', 'overweight',
       'diabetes', 'diagnose_strk', 'smoker', 'high_bp', 'high_chol',
       'heavy_drinker', 'exercise', 'fruit', 'vege'],
      dtype='object')

In [23]:
hr_df

,heart_attack_disease,sex,age,race_ethn,bmi,overweight,diabetes,diagnose_strk,smoker,high_bp,high_chol,heavy_drinker,exercise,fruit,vege
0,0.0,1.0,70.0,0.0,15.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0
1,1.0,1.0,67.0,1.0,28.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
2,1.0,1.0,72.0,1.0,28.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,62.0,0.0,33.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
4,1.0,0.0,76.0,5.0,29.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438688,0.0,1.0,30.0,1.0,25.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
438689,0.0,0.0,80.0,5.0,28.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
438690,0.0,0.0,54.0,1.0,31.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
438691,0.0,0.0,67.0,1.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0


In [24]:
hr_df.heart_attack_disease.value_counts()

0.0    347607
1.0     33540
Name: heart_attack_disease, dtype: int64

In [25]:
engine = create_engine('sqlite:////Users/hannahkim/project/project4/heart_disease.db')
connection = engine.connect()

hr_df.to_sql('heart_disease', con = engine, index = False, if_exists = 'replace')

381147

In [26]:
hr_df.to_csv('cleaned_dataset.csv', index = False)

In [27]:
hr_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 381147 entries, 0 to 438692
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   heart_attack_disease  381147 non-null  float64
 1   sex                   381147 non-null  float64
 2   age                   381147 non-null  float64
 3   race_ethn             381147 non-null  float64
 4   bmi                   381147 non-null  float64
 5   overweight            381147 non-null  float64
 6   diabetes              381147 non-null  float64
 7   diagnose_strk         381147 non-null  float64
 8   smoker                381147 non-null  float64
 9   high_bp               381147 non-null  float64
 10  high_chol             381147 non-null  float64
 11  heavy_drinker         381147 non-null  float64
 12  exercise              381147 non-null  float64
 13  fruit                 381147 non-null  float64
 14  vege                  381147 non-null  float64
dtype

In [28]:
hr_df.describe()

,heart_attack_disease,sex,age,race_ethn,bmi,overweight,diabetes,diagnose_strk,smoker,high_bp,high_chol,heavy_drinker,exercise,fruit,vege
count,381147.000000,381147.000000,381147.000000,381147.000000,381147.000000,381147.000000,381147.000000,381147.000000,381147.000000,381147.000000,381147.000000,381147.000000,381147.000000,381147.000000,381147.000000
mean,0.087998,0.520280,54.613915,0.770634,28.677563,0.715690,0.152975,0.043668,0.407635,0.592003,0.355983,0.060932,0.739478,0.637851,0.823593
std,0.283292,0.499589,17.581666,1.505965,6.472687,0.451086,0.359964,0.204356,0.491395,0.491463,0.478811,0.239206,0.438920,0.480622,0.381167
min,0.000000,0.000000,18.000000,0.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,40.000000,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,1.000000,57.000000,0.000000,28.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000
75%,0.000000,1.000000,69.000000,1.000000,32.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,80.000000,5.000000,99.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
